In [1]:
import findspark
from pyspark.sql import SparkSession
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import configparser
import sql_queries
import pandas as pd
import pandas.io.sql as sqlio
import logging as log


Building spark session

In [2]:
findspark.init()

ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).

In [ ]:
spark = SparkSession.builder \
        .appName('postgres-cnn') \
        .master('spark://0.0.0.0:7077') \
        .config('spark.jars', 'C:\\Users\\pit\\Google Drive\\Udacity\\Capstone Project\\jdbc\\postgresql-42.2.5.jar',) \
        .getOrCreate()

Checking if connection is successful

In [ ]:
df = spark.read \
    .format('jdbc') \
    .option('url', 'jdbc:postgresql://localhost:5432/cnn') \
    .option('dbtable', 'ccy_crypto_stage') \
    .option('user', 'postgres') \
    .option('password', '') \
    .option('driver', 'org.postgresql.Driver') \
    .load()

df.printSchema()

Reading all tables

In [4]:
config = configparser.ConfigParser()
config.read('C:\\Users\\pit\\Google Drive\\Udacity\\Capstone Project\\cfg.cfg')

conn = psycopg2.connect(
    host=config['POSTGRES']['HOST'],
    user=config['POSTGRES']['USER'],
    database=config['POSTGRES']['DATABASE'],
    password=config['POSTGRES']['PASS'],
    port=config['POSTGRES']['PORT']
)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()

table_names = sqlio.read_sql_query(sql_queries.get_table_names, conn)
if table_names.head() is not None:
    print('Table fetched correctly')
else:
    print('Table empty')
    log.info('Table empty')

tn = table_names[table_names.table_name.str.contains('dim*')]
tn.head()

Table fetched correctly


,table_name
3,dim_ccy_aed
4,dim_ccy_aud
5,dim_ccy_bhd
6,dim_ccy_bnd
7,dim_ccy_brl


Check if we have all tables we can run psql query in our postgres
```sql
SELECT COUNT(table_name)
FROM information_schema.tables
WHERE table_name ~ '^dim*';
```
and compare to fetched results

In [5]:
dim_tabs =2055
if tn.count().values[0] == dim_tabs:
    print('All table names fetched')
else:
    print('Tables missing!')
    log.info('Tables missing!')


All table names fetched


Calculating day-to-day delta of rates movement

In [ ]:
spark_dim = spark.read \
    .format('jdbc') \
    .option('url', 'jdbc:postgresql://localhost:5432/cnn') \
    .option('dbtable', 'dim_ccy_aed') \
    .option('user', 'postgres') \
    .option('password', '') \
    .option('driver', 'org.postgresql.Driver') \
    .load()

spark_dim.printSchema()
spark_dim.columns
print(spark_dim.count())
# spark_dim.select('closure_rate').describe().show()

spark.stop()